##Connecting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

##Meta creation

In [ ]:
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm

data_folder = Path('/content/val') # specify train or validation folder
df = []

for path, _, files in tqdm(os.walk(data_folder)):
    for file in files:
      
      folder = path.split('/')[-3]
      type_ = path.split('/')[-2]
      id = path.split('/')[-1]
      df.append([folder, type_, id, file])

df = pd.DataFrame(df, columns = ['folder', 'type', 'id', 'path'])
df.to_csv('meta_val.csv')

0it [00:00, ?it/s]

##Meta Demonstration

In [ ]:
import pandas as pd

demonstration = pd.read_csv('/content/meta_train.csv')
demonstration.head()

,Unnamed: 0,folder,type,id,path
0,0,train,noisy,6625,6625_70995_6625-70995-0008.npy
1,1,train,noisy,6625,6625_70995_6625-70995-0004.npy
2,2,train,noisy,6625,6625_39674_6625-39674-0024.npy
3,3,train,noisy,6625,6625_39680_6625-39680-0034.npy
4,4,train,noisy,6625,6625_70995_6625-70995-0006.npy


##Sound Dataset

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import os
from tqdm.notebook import tqdm

class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return augmented['image'][0].unsqueeze(0).float(), torch.Tensor([label]).float()
        

##Separation of meta: Training (80%), validation (20%)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/meta_train.csv', index_col=0) # Specify mata train

df = df.sort_values(by=['id'], ignore_index=True)

train_size = int(0.8 * df.shape[0])
val_size = df.shape[0] - train_size

border_id = df['id'][train_size]

train_df = df[df['id'] < border_id]
val_df = df[df['id'] >= border_id]

##Train and Validation step

In [ ]:
def train_step(images, labels, architecture, criterion, optimizer):
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    images, labels = images.to(device), labels.to(device)
    architecture.train() # enforce training regime

    pred_image, pred_logits = architecture(images)
    loss = criterion(pred_logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data.cpu().numpy()

def val_step(images, labels, architecture, criterion, optimizer):
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    images, labels = images.to(device), labels.to(device)
    architecture.eval() # enforce evaluation regime

    pred_image, pred_logits = architecture(images)
    loss = criterion(pred_logits, labels)


    return loss.data.cpu().numpy()

##Initialize Dataset, Dataloader

In [ ]:
from torch.utils.data import DataLoader, Dataset
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import torch

train_transform = albu.Compose([albu.PadIfNeeded(480, 80),
            albu.RandomCrop(480, 80),
            albu.Resize(576, 96),
            ToTensorV2()])

train_dataset = SoundDataset(train_df, '/content/data', transforms = train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=32,
                                             num_workers=1, shuffle = True)
val_dataset = SoundDataset(val_df, '/content/data', transforms = train_transform)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=32,
                                             num_workers=1, shuffle = False)

In [ ]:
!pip install segmentation-models-pytorch

##Train

In [ ]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import torch.optim as optim

device = ('cuda' if torch.cuda.is_available() else 'cpu')
params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)

model = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(50):
    epoch_loss_train = 0
    epoch_loss_val = 0
    for X_batch, y_batch in tqdm(train_loader):

        loss_train = train_step(X_batch, y_batch, model, criterion, optimizer)
        
        epoch_loss_train += loss_train     

    for X_batch, y_batch in tqdm(val_loader):

        loss_val = val_step(X_batch, y_batch, model, criterion, optimizer)
        
        epoch_loss_val += loss_val 


    print(f'Epoch {epoch+0:03}: | Train_Loss: {epoch_loss_train/len(train_loader):.5f}')
    print(f'Epoch {epoch+0:03}: | Val_Loss: {epoch_loss_val/len(val_loader):.5f}')

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 000: | Train_Loss: 0.52095
Epoch 000: | Val_Loss: 0.50736


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 048: | Train_Loss: 0.50522
Epoch 048: | Val_Loss: 0.51245


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 049: | Train_Loss: 0.50577
Epoch 049: | Val_Loss: 0.50756


##Save weights

In [ ]:
import torch

torch.save(model.state_dict(), '/content/gdrive/MyDrive/unet_21.pth')

##Test

In [ ]:
!pip install torchmetrics
!pip install segmentation_models_pytorch

In [17]:
import pandas as pd
import albumentations as albu
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics import MeanSquaredError
from torchmetrics import Accuracy
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader, Dataset
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import numpy as np
import os

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

In [18]:
class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return augmented['image'][0].unsqueeze(0).float(), torch.Tensor([label]).float()
        

In [19]:
test_df = pd.read_csv('/content/meta_val.csv') # Specify test meta
test_transform = albu.Compose([albu.PadIfNeeded(480, 80),
            albu.RandomCrop(480, 80),
            albu.Resize(576, 96),
            ToTensorV2()])

test_dataset = SoundDataset(test_df, '/content/data', transforms = test_transform) # Specify data folder
test_loader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=1,
                                             num_workers=1)

device = ('cuda' if torch.cuda.is_available() else 'cpu')

params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)
model = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params).to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/unet_21.pth')) # Specify weights
model.eval()

metric = Accuracy(0.5).to(device)

accuracy_metric = list()

with torch.no_grad():
  for x, y in tqdm(test_loader):

      x, y = x.to(device), y.to(device)
      pr_image, y_pred = model(x)
      accuracy_metric.append(metric(y_pred, y.type(torch.int64)).to('cpu').detach().numpy())

print('Accuracy = ', np.mean(accuracy_metric))

  0%|          | 0/2000 [00:00<?, ?it/s]

Accuracy =  0.991
